The goal is to prepare the unique eigenvector $|x\rangle$ of a diagonal unitary $U$, whose eigenphase $\theta(x)$ satisfies $e^{2\pi i \theta(x)} = e^{2\pi i t}$, using **Quantum Phase Estimation** and **Grover's amplification**.

First we want to simulate a scenario where:

$$
U |x\rangle = e^{2\pi i \theta(x)} |x\rangle
$$

for some known eigenstate $|x\rangle$, and estimate the value of $\theta(x)$ using QPE.

So we:

1. Randomly pick a known eigenvector $|x\rangle$,
2. Retrieve its associated eigenphase $\theta(x)$,
3. Compute the **binary approximation** that QPE should return.

* We want to **simulate the output of QPE**, which estimates $\theta(x)$ to **$d$** bits of precision.
* Since QPE returns a $d$-bit binary approximation of $\theta$, we multiply:

  $$
  \theta(x) \cdot 2^d
  $$

  and take the **integer part**. That gives you:

  $$
  \tilde{k} = \lfloor 2^d \cdot \theta(x) \rfloor \in \{0, \dots, 2^d - 1\}
  $$
* `format(..., f'0{d}b')` converts that integer to a **binary string of length `d`**, padding with leading 0s if necessary.

In [5]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import Diagonal, QFT
from qiskit.quantum_info import Statevector
from collections import Counter
import numpy as np

# Define system and ancilla qubit numbers
n = 3  # system qubits
d = 3  # ancilla qubits
N = 2**n

# Random eigenphases θ(x) ∈ [0,1), unique
theta_list = np.sort(np.random.choice(np.linspace(0, 1, 2**d, endpoint=False), size=N, replace=False))
eigenvalues = np.exp(2j * np.pi * theta_list)

# Pick a target eigenstate x such that U|x> = e^{2πiθ(x)}|x>
target_index = np.random.randint(N)
target_phase = theta_list[target_index]
target_bin = format(int(target_phase * 2**d), f'0{d}b')

print(f"Target index: {target_index}, Target θ(x): {target_phase:.3f} → bin: {target_bin}")

Target index: 2, Target θ(x): 0.250 → bin: 010


## Step 1: Represent $U$ using `Diagonal` gate and construct its powers

If you're given the eigenvalues of $U$, e.g., a length $2^n$ list:

$$
U = \text{diag}(e^{2\pi i \theta(0)}, \dots, e^{2\pi i \theta(2^n - 1)})
$$

For **Quantum Phase Estimation (QPE)**, we need powers of controlled-`U`, `U^1`, `U^2`, ..., `U^{2^{d-1}}`. You can do this using Qiskit’s `ControlledGate`:



In [6]:
controlled_U_gates = []
for k in range(d):
    eigs = np.power(eigenvalues, 2**k)
    U_power = Diagonal(eigs)
    controlled_U = U_power.control()
    controlled_U_gates.append(controlled_U)

## Step 2: Apply Quantum Phase Estimation (QPE)

This step estimates $\theta(x)$ using $d$ ancilla qubits and $n$ system qubits:

In [7]:
qc = QuantumCircuit(d + n)
qc.h(range(d))         # Hadamard on ancilla
qc.h(range(d, d + n))  # Superposition on system qubits

# Controlled-U^2^k
for i in range(d):
    qc.append(controlled_U_gates[i], [i] + [j + d for j in range(n)])

# Inverse QFT
qc.append(QFT(d, inverse=True).to_gate(label='QFT†'), range(d))

## Step 3: Grover Oracle — Mark the index with phase close to $t$

You now create an oracle that flags the index whose estimated phase is **closest to** $t$. One way is to measure the ancilla (phase) register **in simulation**, check which outcome is closest to $t$, and **mark** this bitstring in the Grover circuit.


In [8]:
# Oracle: flips phase if state matches target_phase_bin
oracle = QuantumCircuit(d + n)
oracle_bits = [i for i, b in enumerate(target_bin) if b == '0']

# Conditional X gates
if oracle_bits:
    oracle.x(oracle_bits)

oracle.h(d - 1)
oracle.mcx(list(range(d - 1)), d - 1)
oracle.h(d - 1)

if oracle_bits:
    oracle.x(oracle_bits)

## Step 5: Run Grover’s Amplification

Grover amplification boosts the amplitude of the marked state. Construct the diffuser and repeat the oracle + diffuser process a few times.


In [9]:
def diffuser(qc, qubits):
    qc.h(qubits)
    qc.x(qubits)
    qc.h(qubits[-1])
    qc.mcx(qubits[:-1], qubits[-1])
    qc.h(qubits[-1])
    qc.x(qubits)
    qc.h(qubits)

grover = QuantumCircuit(d + n)
grover.compose(oracle, inplace=True)
diffuser(grover, list(range(d)))

## Final Step: Full circuit to prepare |x⟩

Put all the pieces together: QPE → Grover iterations → measurement

Question: I am not sure why Grover is not amplifying the target bin correctly?

In [10]:
# Compose full circuit
full_circuit = qc.copy()
for _ in range(20):
    full_circuit = full_circuit.compose(grover)

# --- Simulate ---
sv = Statevector.from_instruction(full_circuit)
probs = sv.probabilities_dict()

# Display full amplitudes
print("\nFull bitstring amplitudes:")
for b, p in sorted(probs.items(), key=lambda x: -x[1]):
    if p > 1e-6:
        print(f"  {b} → {p:.4f}")

# Analyze system register
system_probs = Counter()
for bitstring, p in probs.items():
    system = bitstring[d:]  # system register is last n bits
    system_probs[system] += p

print("\nTop outcomes for system register:")
for x, p in sorted(system_probs.items(), key=lambda x: -x[1])[:5]:
    print(f"  |{x}⟩ → {p:.4f}")


Full bitstring amplitudes:
  011011 → 0.0825
  111111 → 0.0825
  110110 → 0.0825
  100100 → 0.0825
  001001 → 0.0825
  000000 → 0.0825
  101101 → 0.0825
  011010 → 0.0161
  001010 → 0.0161
  010001 → 0.0161
  100010 → 0.0161
  111010 → 0.0161
  010000 → 0.0161
  010111 → 0.0161
  110010 → 0.0161
  010100 → 0.0161
  010101 → 0.0161
  010110 → 0.0161
  000010 → 0.0161
  101010 → 0.0161
  010011 → 0.0161
  010010 → 0.0122
  111100 → 0.0044
  001000 → 0.0044
  001110 → 0.0044
  011111 → 0.0044
  110101 → 0.0044
  111001 → 0.0044
  001011 → 0.0044
  101100 → 0.0044
  110100 → 0.0044
  111000 → 0.0044
  000100 → 0.0044
  000111 → 0.0044
  001101 → 0.0044
  011000 → 0.0044
  011110 → 0.0044
  100001 → 0.0044
  100011 → 0.0044
  111101 → 0.0044
  000001 → 0.0044
  000101 → 0.0044
  000110 → 0.0044
  011101 → 0.0044
  110011 → 0.0044
  100101 → 0.0044
  111110 → 0.0044
  001100 → 0.0044
  110001 → 0.0044
  001111 → 0.0044
  111011 → 0.0044
  011100 → 0.0044
  100000 → 0.0044
  101111 → 0.0044


In [11]:
print(f"Target phase: {target_phase:.3f}, Target bin: {target_bin}")
print(f"Oracle flips phase when ancilla = {target_bin}")
print(f"X-gates applied to qubits: {[i for i, b in enumerate(target_bin) if b == '0']}")

Target phase: 0.250, Target bin: 010
Oracle flips phase when ancilla = 010
X-gates applied to qubits: [0, 2]
